In [1]:
import api_calls as api
import pandas as pd
import regex as re
from pathlib import Path
import os

In [2]:
TESTING_DATA = Path("./data/test_data/")
RESULTS = Path("./data/results/")

In [3]:
test_data = pd.read_csv(TESTING_DATA/"test_data.csv")

In [2]:
system_0 = "you are an economist. label the following sentence as hawkish, dovish or neutral and provide a reason for your label."
system_1 = "you are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it is corresponding to tightening of the monetary policy, 'DOVISH' if it is corresponding to easing of the monetary policy, or 'NEUTRAL' if the stance is neutral. Provide the label in the first line and provide a short explanation in the second line."

In [6]:
model_answers = {"gpt-3.5-turbo":None}
for model in model_answers.keys():
    model_answers[model] = await api.get_multiple_api_calls(system_1, list(test_data.iloc[:,0]), model)
await api.close_session()

2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:24 [info     ] got response                   response_status=200
2023-10-07 21:41:25 [info     ] got response                   response_status=200
2023-10-07 21:41:25 [info     ] got response                   response_status=200
2023-10-07 21:41:25 [info     ] got response                   response_status=200
2023

CancelledError: 

In [7]:
await api.close_session()

In [33]:
for model, answers in model_answers.items():
        test_data[model] = answers

In [34]:
test_data.to_csv(RESULTS/"test_results_large_raw_system_1.csv")

In [35]:
def find_label(s):
    match = re.search(r'\b(hawkish|dovish|neutral)\b', s, re.IGNORECASE)
    if match:
        return match.group(0).lower()  # return the found label in lowercase
    else:
        return None

In [39]:
for i in range(test_data.shape[0]):
        test_data.iloc[i,3] = find_label(test_data.iloc[i,3])

In [43]:
for i in range(test_data.shape[0]):
    if test_data.iloc[i,2] == "neutral":
        test_data.iloc[i,2] = 2
    elif test_data.iloc[i,2] == "hawkish":
        test_data.iloc[i,2] = 1
    else:
        test_data.iloc[i,2] = 0

In [44]:
test_data.to_csv(RESULTS/"test_results_large_filtered_system_1.csv")

In [3]:
success = 0
for i, j in zip(test_data["label"], test_data[os.getenv("FINE_TUNE_LARGE")]):
    if i == j:
        success += 1

NameError: name 'test_data' is not defined

In [55]:
success/test_data.shape[0]*100

75.41703248463566

In [69]:
list(test_data.iloc[:10,0])

['However, in light of the projected persistence of slack in labor and product markets and the anticipated stability in long-term inflation expectations, the increase in inflation was expected to be mostly transitory if oil and other commodity prices did not rise significantly further.',
 'Broad equity price indexes fell sharply over the intermeeting period on net.',
 'For a variety of reasons, some emerging-market economies have resisted upward pressures on their exchange rates, even if that resistance requires buying large quantities of dollars to keep their currencies from appreciating.',
 'The extraordinary achievement of 1996, of course, was reaching such low levels of unemployment and inflation at the same time.',
 'A few participants particularly stressed the high uncertainty associated with the expected future path of the unemployment rate and commented that the unemployment rate could rise by considerably more than in the staff forecast.',
 'All members agreed that the risks t

In [5]:
import tokenisation as tkn
import regex as re

In [6]:
pc = tkn.get_press_conferences("https://www.federalreserve.gov/mediacenter/files/FOMCpresconf20230503.pdf")
pc_split = tkn.extract_lines(pc, ["CHAIR POWELL", "CHAIRMAN POWELL"])
pc_tokenised = tkn.get_split_press_conference(pc_split)

In [16]:
test_1 = await api.get_multiple_api_calls(system_0, pc_tokenised, "gpt-4")

2023-10-09 15:08:25 [info     ] got response                   response_status=200
2023-10-09 15:08:25 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023-10-09 15:08:26 [info     ] got response                   response_status=200
2023

In [17]:
def find_label(s):
    match = re.search(r'\b(hawkish|dovish|neutral)\b', s, re.IGNORECASE)
    if match:
        return match.group(0).lower()  # return the found label in lowercase
    else:
        return None
test_2 = []
for i in test_1:
        test_2.append(find_label(i))
test_3 = []
for i in test_2:
    if i == "neutral":
        test_3.append(2)
    elif i == "hawkish":
        test_3.append(1)
    else:
        test_3.append(0)

In [18]:
test_3.count(1)

56

In [19]:
test_3.count(2)


270

In [20]:
test_3.count(0)


41

In [63]:
import re

def extract_lines(script, character_names):
    # Ensure that character_names is a list
    if not isinstance(character_names, list):
        character_names = [character_names]
    
    all_matches = []
    
    for character_name in character_names:
        # Create a regex pattern with optional spaces and newlines for the character's name
        spaced_character = '[\\s\\n]*'.join(character_name)
        pattern = f"{spaced_character}[\\.\\:][\\s\\n]*(.*?)(?=(?:[A-Z][A-Z\\s\\n]*[A-Z][\\.\\:])|$)"
        
        # Use regex findall to extract all matches
        matches = re.findall(pattern, script, re.DOTALL)
        
        # Clean the matches and add them to the all_matches list
        all_matches.extend([match.strip().replace('\n', ' ') for match in matches])

    return all_matches

# Example usage:
# lines = extract_lines(script_content, ["CHARACTER1", "CHARACTER2"])



In [64]:
extract_lines(pc, "CHAIR POWELL")

['Good afternoon.  Before discussing today’ s meeting, let me comment briefly on recent developments in the banking sector.  Conditions in that sector have broadly improved since early March, and the U.S banking system is sound and resilient.  We will continue to monitor conditions in the  sector.  We are committed to learning the  right  lessons from this episode and will work to prevent events like these from happening again.  As a first step in that process, last week we released Vice Chair for Supervision Barr ’s Review of the Federal Reserve’ s Super vision and Regulation of Silicon Valley Bank.  The review ’s findings underscore the need to address our rules and supervisory practices to make for a stronger and more resilient banking system, and I am confident that we will do so. From the perspective of  monetary policy, our focus remains squarely on our dual mandate to promote maximum employment and stable prices for the American people.  My colleagues and I understand the hardsh